In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.plotly as py
from sklearn.preprocessing import StandardScaler
import scipy
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


C:\Users\dubey\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Importing the Data 
train =  pd.read_csv("D:\\Kaggle Projects\\Loan\\train.csv")
test =  pd.read_csv("D:\\Kaggle Projects\\Loan\\test.csv")

In [3]:
train['LoanAmount'].median()

128.0

In [4]:
train['LoanAmount'].fillna(128, inplace=True)

In [5]:
test['LoanAmount'].median()

125.0

In [6]:
test['LoanAmount'].fillna(125, inplace=True)

In [7]:
train['Loan_Amount_Term'].median()

360.0

In [8]:
train['Loan_Amount_Term'].fillna(360, inplace=True)

In [9]:
test['Loan_Amount_Term'].median()

360.0

In [10]:
test['Loan_Amount_Term'].fillna(360, inplace=True)

In [11]:
train.loc[(train.Gender== 'Male') ,'Gender' ] = 1
train.loc[(train.Gender== 'Female') ,'Gender' ] = 0

In [12]:
test.loc[(test.Gender== 'Male') ,'Gender' ] = 1
test.loc[(test.Gender== 'Female') ,'Gender' ] = 0

In [13]:
train['Gender'].fillna(1, inplace=True)
test['Gender'].fillna(1, inplace=True)

In [14]:
train['Married'].fillna(0, inplace=True)

In [15]:
test['Married'].fillna(0, inplace=True)

In [16]:
train.loc[(train.Married== 'No') ,'Married' ] = 1
train.loc[(train.Married== 'Yes') ,'Married' ] = 0

In [17]:
test.loc[(test.Married== 'No') ,'Married' ] = 1
test.loc[(test.Married== 'Yes') ,'Married' ] = 0

In [18]:
train.loc[(train.Education== 'Graduate') ,'Education' ] = 1
train.loc[(train.Education== 'Not Graduate') ,'Education' ] = 0

In [19]:
test.loc[(test.Education== 'Graduate') ,'Education' ] = 1
test.loc[(test.Education== 'Not Graduate') ,'Education' ] = 0

In [20]:
train.loc[(train.Loan_Status== 'Y') ,'Loan_Status' ] = 1
train.loc[(train.Loan_Status== 'N') ,'Loan_Status' ] = 0

In [21]:
train.loc[(train.Property_Area== 'Urban') ,'Property_Area' ] = 0
train.loc[(train.Property_Area== 'Rural') ,'Property_Area' ] = 1
train.loc[(train.Property_Area== 'Semiurban') ,'Property_Area' ] = 2

In [22]:
test.loc[(test.Property_Area== 'Urban') ,'Property_Area' ] = 0
test.loc[(test.Property_Area== 'Rural') ,'Property_Area' ] = 1
test.loc[(test.Property_Area== 'Semiurban') ,'Property_Area' ] = 2

In [23]:
train['Property_Area'] = train['Property_Area'].apply(int)
test['Property_Area'] = test['Property_Area'].apply(int)

In [24]:
train.loc[(train.Self_Employed== 'No') ,'Self_Employed' ] = 0
train.loc[(train.Self_Employed== 'Yes') ,'Self_Employed' ] = 1

In [25]:
test.loc[(test.Self_Employed== 'No') ,'Self_Employed' ] = 0
test.loc[(test.Self_Employed== 'Yes') ,'Self_Employed' ] = 1

In [26]:
train['Self_Employed'].fillna(0, inplace=True)

In [27]:
test['Self_Employed'].fillna(0, inplace=True)

In [28]:
train.loc[(train.Dependents== '3+') ,'Dependents' ] = 3
train['Dependents'].fillna(4, inplace=True)

In [29]:
test.loc[(test.Dependents== '3+') ,'Dependents' ] = 3
test['Dependents'].fillna(4, inplace=True)

In [30]:
train['Dependents'] = train['Dependents'].apply(int)
test['Dependents'] = test['Dependents'].apply(int)

In [31]:
train.loc[(train.Dependents== 4) ,'Dependents' ] = 0
test.loc[(test.Dependents== 4) ,'Dependents' ] = 0

In [32]:
train['Credit_History'].fillna(1, inplace=True)
test['Credit_History'].fillna(1, inplace=True)

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null object
Dependents           614 non-null int64
Education            614 non-null object
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           614 non-null float64
Loan_Amount_Term     614 non-null float64
Credit_History       614 non-null float64
Property_Area        614 non-null int64
Loan_Status          614 non-null object
dtypes: float64(4), int64(5), object(4)
memory usage: 62.4+ KB


In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               367 non-null int64
Married              367 non-null object
Dependents           367 non-null int64
Education            367 non-null object
Self_Employed        367 non-null int64
ApplicantIncome      367 non-null int64
CoapplicantIncome    367 non-null int64
LoanAmount           367 non-null float64
Loan_Amount_Term     367 non-null float64
Credit_History       367 non-null float64
Property_Area        367 non-null int64
dtypes: float64(3), int64(6), object(3)
memory usage: 34.5+ KB


In [35]:
y_train = train['Loan_Status']
Y_train = list(y_train.values)

In [36]:
x_train = train.drop('Loan_Status', 1)

In [37]:
LoanID1 = test['Loan_ID']

In [38]:
x_train['Loan_ID'] = x_train['Loan_ID'].map(lambda x: x.lstrip('LP'))
test['Loan_ID'] = test['Loan_ID'].map(lambda x: x.lstrip('LP'))

In [39]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
scaler = StandardScaler().fit(test)
test = scaler.transform(test)

In [40]:
clf = LogisticRegression(solver='lbfgs' , multi_class='multinomial', C=100, tol=0.08)
clf.fit(x_train, Y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.08, verbose=0, warm_start=False)

In [42]:
# Printing the Model score from both train and test data 
print(clf.score(x_train, Y_train))

0.811074918567


In [43]:
y_test = clf.predict(test)


In [44]:
y_test = pd.DataFrame(y_test,columns=['Values'],index=LoanID1)


In [46]:
y_test.to_csv('D:\\Kaggle Projects\\Loan\\submission_model_final.csv')

In [ ]:
#TRYING WITH OTHER MODELS AS WELL

In [51]:
train_X = np.array(x_train).astype('float')
test_X = np.array(test).astype('float')

In [52]:
train_y = np.array(Y_train)

In [53]:
xgtrain = xgb.DMatrix(train_X, label=train_y)

In [54]:
xgtest = xgb.DMatrix(test_X)

In [49]:
params = {}

In [50]:
params["objective"] = "reg:linear"
params["eta"] = 0.05
params["max_depth"] = 5
params["learning_rate"] = 0.1

params["seed"] = 0
plst = list(params.items())
num_rounds = 1000

In [55]:
model = xgb.train(plst, xgtrain, num_rounds)

In [56]:
pred_test_y = model.predict(xgtest)

In [58]:
Y_test = pd.DataFrame(pred_test_y,columns=['Loan_Status'],index=LoanID1)

In [60]:
Y_test.to_csv('D:\\Kaggle Projects\\Loan\\submission_model.csv')